# Part 2 – DeFi Protocol Interaction

This notebook is a thin demo on top of the Python package in `src/lido_takehome/`.
All on-chain logic (ABI loading, pool helpers, impersonation, withdrawal) lives in
`curve.py`; here we just:

1. Connect to Ethereum mainnet and a local Anvil mainnet fork.
2. Inspect the Curve USDC/crvUSD pool composition.
3. Check the LP position of a real LP whale (from Etherscan).
4. On the fork, impersonate that whale and withdraw a fraction of their LP
   position as single-sided USDC via `remove_liquidity_one_coin`.
5. Report the resulting USDC received and the implied USDC per LP token.

The goal is to show a realistic “what would happen if this LP whale exited into
USDC?” scenario in a safe, reproducible environment.

## Load modules and system paths

Import the package (`lido_takehome`) and make sure the project root is on
`sys.path`. The Web3 configuration (`get_web3_mainnet`, `get_web3_local`) and
all Curve helpers are defined in `src/lido_takehome`.

In [1]:
import os
import sys
from pathlib import Path

from lido_takehome.config import get_web3_mainnet, get_web3_local

from lido_takehome.config import get_web3_local
from lido_takehome.curve import get_pool_contract, get_pool_balances

from lido_takehome.config import CURVE_LP_WHALE
from lido_takehome.curve import get_lp_balance, withdraw_usdc_single_sided


from web3 import Web3


cwd = Path(os.getcwd()).resolve()

# If you're running the notebook from the repo root or from notebooks/
if (cwd / "src").exists():
    project_root = cwd
else:
    project_root = cwd.parent  # e.g. when cwd == .../notebooks

sys.path.append(str(project_root / "src"))

print("Using project_root:", project_root)
print("In sys.path:", project_root / "src" in map(Path, map(str, sys.path)))

Using project_root: /Users/kb/Dev/Projects/Lido_QuantAnalytics_TakeHome
In sys.path: True


## Test connectivity to mainnet and local fork

Connect to:
- Ethereum mainnet via Alchemy (`get_web3_mainnet`)
- A local Anvil mainnet fork (`get_web3_local`)

and print basic connectivity info. This is just a sanity check that RPC and
environment variables are set up correctly.

In [2]:
w3 = get_web3_mainnet()
w3_local = get_web3_local()

print("Mainnet:", w3.is_connected(), "chain_id:", w3.eth.chain_id)
print("Fork:", w3_local.is_connected(), "chain_id:", w3_local.eth.chain_id)

Mainnet: True chain_id: 1
Fork: True chain_id: 1


### Inspect the Curve USDC/crvUSD pool

Use the helper functions from `curve.py` to:

- Instantiate the Curve USDC/crvUSD pool contract on the fork.
- Verify the underlying `coins(0)` / `coins(1)` addresses.
- Read the pool balances and convert them to human-readable units using the
  ERC-20 `decimals()` metadata.

This confirms that:
- the correct pool ABI is loaded, and
- the pool state on the fork matches mainnet at the fork block.

In [3]:
w3_local = get_web3_local()

pool = get_pool_contract(w3_local)
print(pool.functions.coins(0).call())
print(pool.functions.coins(1).call())

0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E


In [4]:
Web3.to_checksum_address("0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E")

'0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'

In [5]:
print(get_pool_balances(w3_local))

(49813974.662407, 49221833.26968017)


### Check LP whale position on the fork

`CURVE_LP_WHALE` is configured in `config.py` and was chosen from the LP token
Holders tab on Etherscan for the USDC/crvUSD pool.

We now check:
- the pool’s current composition, and
- the raw LP token balance of this whale on the fork (`get_lp_balance`).

This ensures we are impersonating a real, sizable LP position.

In [6]:
w3_local = get_web3_local()
print(get_pool_balances(w3_local))

lp_balance_raw = get_lp_balance(w3_local, CURVE_LP_WHALE)
lp_used, usdc_before, usdc_after = withdraw_usdc_single_sided(
    w3_local,
    CURVE_LP_WHALE,
    fraction_of_balance=0.02,
)
usdc_received = usdc_after - usdc_before
usdc_received

(49813974.662407, 49221833.26968017)


1188680.043035

## Run Part 2 – Impersonate LP whale and withdraw USDC

On the local Anvil fork, we now:

1. Impersonate the LP whale address (`CURVE_LP_WHALE`) using Anvil’s
   `anvil_impersonateAccount`.
2. Top up the whale with a small amount of ETH for gas using
   `anvil_setBalance` (this only affects the local fork).
3. Compute a target LP amount as a fraction of the whale’s LP balance
   (here: 5%).
4. Ask the pool for a quote via `calc_withdraw_one_coin` to derive a minimum
   USDC amount (slippage-protected).
5. Call `remove_liquidity_one_coin` from the impersonated whale.
6. Measure USDC balance before and after the transaction.

The heavy lifting is done by `withdraw_usdc_single_sided` in `curve.py`; the
cell below just calls it and computes the net USDC received.

In [7]:
w3_local = get_web3_local()

# This should now be > 0
get_lp_balance(w3_local, CURVE_LP_WHALE)

lp_used, usdc_before, usdc_after = withdraw_usdc_single_sided(
    w3_local,
    CURVE_LP_WHALE,
    fraction_of_balance=0.05,
)

usdc_out = usdc_after - usdc_before
lp_used, usdc_out

(2848191.34344289, 2912233.746104)

### Results

We report:
- `LP used`: the fraction of the whale’s LP tokens that were burned.
- `USDC out`: the net USDC the whale would receive on the fork.
- `Implied USDC per LP`: an effective “price per LP token” implied by this
  single-sided withdrawal, given the current pool state and slippage settings.

This demonstrates the end-to-end flow the assignment asked for:
impersonating a real LP whale on a mainnet fork and simulating a partial
exit to USDC using the Curve pool’s on-chain interface.

In [8]:
print(f"LP used:   {lp_used:,.2f} LP tokens")
print(f"USDC out:  {usdc_out:,.2f} USDC")
print(f"Implied USDC per LP: {usdc_out / lp_used:,.6f}")

LP used:   2,848,191.34 LP tokens
USDC out:  2,912,233.75 USDC
Implied USDC per LP: 1.022485
